In [1]:
import sys
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import tabulate

## Tiempo de viaje a la mañana

Me traigo los datasets que tienen los tiempo de viaje, pero también me debo traer el dataset con los datos personales (pues hay personas de la FCEN para las cuales no se podía calcular el tiempo de viaje)

In [ ]:
tiempo_cbc = pd.read_csv('../../../assets/silver/data_utilizada/distancia_viaje_cbc_solo_manana.csv')
tiempo_fcen = pd.read_csv('../../../assets/silver/data_utilizada/datos_fcen_tiempo_viaje.csv')
personas = pd.read_csv('../../../assets/silver/data_utilizada/personas_con_target.csv')
personas.head(5)

In [4]:
tiempo_fcen_tarde = pd.read_csv('../../../assets/silver/data_utilizada/datos_fcen_tiempo_viaje_tarde.csv')

In [ ]:
personas.dtypes

In [ ]:
tiempo_cbc.dtypes

In [ ]:
tiempo_fcen.dtypes

In [ ]:
tiempo_cbc['Dni'] = tiempo_cbc['Dni'].astype(str)

In [ ]:
tiempo_cbc.head(5)

In [ ]:
tiempo_fcen.head(5)

In [ ]:
print(tiempo_cbc.shape)
print(tiempo_fcen.shape)
print(personas.shape)
print(tiempo_fcen_tarde.shape)

Nos quedamos con las columnas que contienen le dato del dni (para poder juntarlas) y las que contienen el tiempo de viaje

In [ ]:
tiempo_cbc = tiempo_cbc[['Dni', 'Duracion_manana']]
tiempo_cbc.columns = ['dni', 'duracion_manana_cbc']
tiempo_fcen = tiempo_fcen[['dni', 'duracion_manana']]
tiempo_fcen.columns = ['dni', 'duracion_manana_fcen']

In [ ]:
personas = personas.merge(tiempo_cbc, on='dni', how='left')
personas.shape

In [ ]:
personas = personas.merge(tiempo_fcen, on='dni', how='left')
personas.shape

In [ ]:
personas[['duracion_manana_cbc', 'duracion_manana_fcen']].describe()

In [ ]:
personas[['duracion_manana_cbc', 'duracion_manana_fcen']].isnull().sum()

En principio definimos 0 y más de 4 horas (14400seg) como atipico. Vemos como cambia la distribución si cumplimos con eso

In [ ]:
personas['duracion_manana_cbc'] = personas['duracion_manana_cbc'].replace(0, np.nan)
personas['duracion_manana_fcen'] = personas['duracion_manana_fcen'].replace(0, np.nan)

In [ ]:
personas[['duracion_manana_cbc', 'duracion_manana_fcen']].describe()

In [ ]:
personas[['duracion_manana_cbc', 'duracion_manana_fcen']].isnull().sum()

In [ ]:
# reemplazo todo valor mayor a 14400 por NaN, ya que es un valor que no tiene sentido
personas['duracion_manana_cbc'] = personas['duracion_manana_cbc'].replace(
    personas['duracion_manana_cbc'][personas['duracion_manana_cbc'] > 14400].values, np.nan)
personas['duracion_manana_fcen'] = personas['duracion_manana_fcen'].replace(
    personas['duracion_manana_fcen'][personas['duracion_manana_fcen'] > 14400].values, np.nan)

In [ ]:
personas[['duracion_manana_cbc', 'duracion_manana_fcen']].describe()

In [ ]:
personas[['duracion_manana_cbc', 'duracion_manana_fcen']].isnull().sum()

In [ ]:
personas[personas['duracion_manana_cbc'].isnull() & personas['duracion_manana_fcen'].isnull()].shape

In [ ]:
personas[(~personas['duracion_manana_cbc'].isnull()) & (personas['duracion_manana_fcen'].isnull())].shape

In [ ]:
personas[(personas['duracion_manana_cbc'].isnull()) & (~personas['duracion_manana_fcen'].isnull())].shape

In [ ]:
personas[(~personas['duracion_manana_cbc'].isnull()) & (personas['duracion_manana_fcen'].isnull())].shape

Nosotros siempre que podamos nos vamos a quedar con los datos de la FCEN, pues son más actuales. Como hay 9 casos en donde tenemos la información de tiempo de viaje en el CBC pero no en la FCEN, tomamos esos casos para definir su tiempo de viaje a la facultad. Definimos la columna 'tiempo_de_viaje'

In [ ]:
# defino columna 'tiempo_de_viaje' como la duracion de fcen si no es nulo, sino la de cbc
personas['tiempo_de_viaje'] = personas['duracion_manana_fcen']
personas.loc[personas['tiempo_de_viaje'].isnull(), 'tiempo_de_viaje'] = \
    personas['duracion_manana_cbc'][personas['tiempo_de_viaje'].isnull()]

Tiene sentido porque tiene 9 casos menos de nulos gracias a que completamos con el CBC

In [ ]:
personas[['duracion_manana_cbc', 'duracion_manana_fcen', 'tiempo_de_viaje']].isnull().sum()

In [ ]:
personas[['duracion_manana_cbc', 'duracion_manana_fcen', 'tiempo_de_viaje']].describe()

Nos quedamos con las columnas que nos interesan guardar aparte

In [ ]:
tiempo_viaje = personas[['dni', 'tiempo_de_viaje']].copy()
tiempo_viaje.to_csv('../../../assets/silver/data_utilizada/tiempo_viaje_manana.csv', index=False)

In [ ]:
# Estilo visual
sns.set(style="whitegrid", palette="pastel")

# Tamaño del gráfico
plt.figure(figsize=(8, 5))

# Boxplot
sns.boxplot(x=personas['tiempo_de_viaje'], color="skyblue")

# Título y etiquetas
plt.title('Distribución del tiempo de viaje', fontsize=14)
plt.xlabel('Tiempo de viaje (seg)', fontsize=12)
mean_val = personas['tiempo_de_viaje'].mean()
plt.axvline(mean_val, color='red', linestyle='--', label=f'Media: {mean_val:.1f}')
plt.legend()
median_val = personas['tiempo_de_viaje'].median()
plt.axvline(median_val, color='blue', linestyle='--', label=f'Mediana: {median_val:.1f}')
plt.legend()

# Mostrar
plt.tight_layout()
plt.show()


In [ ]:
# Estilo visual
sns.set(style="whitegrid", palette="pastel")

# Tamaño del gráfico
plt.figure(figsize=(8, 5))

# Histograma con densidad opcional
sns.histplot(data=personas, x='tiempo_de_viaje', bins=30, kde=True, color='skyblue')

# Título y etiquetas
plt.title('Distribución del tiempo de viaje', fontsize=14)
plt.xlabel('Tiempo de viaje (seg)', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)

# Mostrar
plt.tight_layout()
plt.show()

In [ ]:
sns.set(style="whitegrid", palette="pastel")
plt.figure(figsize=(8, 5))

# Histograma con proporciones
sns.histplot(data=personas, x='tiempo_de_viaje', bins=40, kde=False, stat="probability", color='skyblue')

# Título y etiquetas
plt.title('Distribución del tiempo de viaje', fontsize=14)
plt.xlabel('Tiempo de viaje (seg)', fontsize=12)
plt.ylabel('Proporción', fontsize=12)
mean_val = personas['tiempo_de_viaje'].mean()
plt.axvline(mean_val, color='red', linestyle='--', label=f'Media: {mean_val:.1f}')
plt.legend()
median_val = personas['tiempo_de_viaje'].median()
plt.axvline(median_val, color='blue', linestyle='--', label=f'Mediana: {median_val:.1f}')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Estilo visual
sns.set(style="whitegrid", palette="pastel")

# Tamaño del gráfico
plt.figure(figsize=(8, 5))

# Histograma con densidad opcional
sns.histplot(data=personas[personas['target'] == 0], x='tiempo_de_viaje', bins=50, kde=True, color='skyblue')

# Título y etiquetas
plt.title('Distribución del tiempo de viaje sin riesgo de abandono', fontsize=14)
plt.xlabel('Tiempo de viaje (seg)', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)

# Mostrar
plt.tight_layout()
plt.show()

In [ ]:
sns.set(style="whitegrid", palette="pastel")
plt.figure(figsize=(8, 5))

# Histograma con proporciones
sns.histplot(data=personas[personas['target'] == 0], x='tiempo_de_viaje', bins=50, kde=True, stat="probability", color='skyblue')

# Título y etiquetas
plt.title('Distribución proporcional del tiempo de viaje sin riesgo de abandono', fontsize=14)
plt.xlabel('Tiempo de viaje (seg)', fontsize=12)
plt.ylabel('Proporción', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Estilo visual
sns.set(style="whitegrid", palette="pastel")

# Tamaño del gráfico
plt.figure(figsize=(8, 5))

# Histograma con densidad opcional
sns.histplot(data=personas[personas['target'] == 1], x='tiempo_de_viaje', bins=50, kde=True, color='orange')

# Título y etiquetas
plt.title('Distribución del tiempo de viaje con riesgo de abandono', fontsize=14)
plt.xlabel('Tiempo de viaje (seg)', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)

# Mostrar
plt.tight_layout()
plt.show()

In [ ]:
sns.set(style="whitegrid", palette="pastel")
plt.figure(figsize=(8, 5))

# Histograma con proporciones
sns.histplot(data=personas[personas['target'] == 1], x='tiempo_de_viaje', bins=50, kde=True, stat="probability", color='orange')

# Título y etiquetas
plt.title('Distribución proporcional del tiempo de viaje con riesgo de abandono', fontsize=14)
plt.xlabel('Tiempo de viaje (seg)', fontsize=12)
plt.ylabel('Proporción', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
personas['grupo_viaje'] = pd.cut(personas['tiempo_de_viaje'], bins=[0, 3600, 14400])

# Normalizado (proporciones)
prop_df = personas.groupby('grupo_viaje')['target'].value_counts(normalize=True).unstack()

prop_df.rename(columns={0: 'No Abandono', 1: 'Abandono'}, inplace=True)
prop_df_abandono = prop_df['Abandono'] 

plt.figure(figsize=(8, 5))
prop_df_abandono.plot(kind='bar', stacked=False, color=["#6CCDA3"])
plt.title("Distribución de Abandono por tiempo de viaje", fontsize=15)
plt.ylabel("Proporción", fontsize=15)
plt.xlabel("Tiempo de viaje (horas)", fontsize=15)
plt.xticks(rotation=0, fontsize=13)
plt.yticks(fontsize=12)
plt.legend(fontsize=14)
plt.show()

In [ ]:
prop_df_abandono

In [ ]:
# Calcular los conteos de cada valor de target por grupo etario
conteos_df = personas.groupby('grupo_viaje')['target'].value_counts().unstack(fill_value=0)
conteos_df.rename(columns={0: 'No Abandono', 1: 'Abandono'}, inplace=True)

# Graficar como barras apiladas
conteos_df.plot(kind='bar', stacked=False, color=["#5D81E2", "#6CCDA3"])
plt.title("Cantidad de personas por tiempo de viaje y target", fontsize=15)
plt.ylabel("Cantidad", fontsize=15)
plt.xlabel("Tiempo de viaje (horas)", fontsize=15)
plt.legend(title="Target", fontsize=12)
plt.xticks(rotation=0, fontsize=13)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

## Tiempo de viaje a la tarde

In [ ]:
tiempo_fcen_tarde = tiempo_fcen_tarde[['dni', 'duracion_tarde']]
tiempo_fcen_tarde.columns = ['dni', 'duracion_tarde_fcen']

In [ ]:
personas = personas.merge(tiempo_fcen_tarde, on='dni', how='left')
personas.shape

In [ ]:
personas['duracion_tarde_fcen'] = personas['duracion_tarde_fcen'].replace(
    personas['duracion_tarde_fcen'][personas['duracion_tarde_fcen'] > 14400].values, np.nan)

In [ ]:
# Estilo visual
sns.set(style="whitegrid", palette="pastel")

# Tamaño del gráfico
plt.figure(figsize=(8, 5))

# Boxplot
sns.boxplot(x=personas['duracion_tarde_fcen'], color="skyblue")

# Título y etiquetas
plt.title('Distribución del tiempo de viaje', fontsize=14)
plt.xlabel('Tiempo de viaje (seg)', fontsize=12)
mean_val = personas['duracion_tarde_fcen'].mean()
plt.axvline(mean_val, color='red', linestyle='--', label=f'Media: {mean_val:.1f}')
plt.legend()
median_val = personas['duracion_tarde_fcen'].median()
plt.axvline(median_val, color='blue', linestyle='--', label=f'Mediana: {median_val:.1f}')
plt.legend()

# Mostrar
plt.tight_layout()
plt.show()


In [ ]:
# Estilo visual
sns.set(style="whitegrid", palette="pastel")

# Tamaño del gráfico
plt.figure(figsize=(8, 5))

# Histograma con densidad opcional
sns.histplot(data=personas, x='duracion_tarde_fcen', bins=30, kde=True, color='skyblue')

# Título y etiquetas
plt.title('Distribución del tiempo de viaje', fontsize=14)
plt.xlabel('Tiempo de viaje (seg)', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)

# Mostrar
plt.tight_layout()
plt.show()

In [ ]:
personas['grupo_viaje_tarde'] = pd.cut(personas['duracion_tarde_fcen'], bins=[0, 3600, 14400])

# Normalizado (proporciones)
prop_df = personas.groupby('grupo_viaje_tarde')['target'].value_counts(normalize=True).unstack()

prop_df.plot(kind='bar', stacked=False)
plt.title("Distribución del target por tiempo de viaje")
plt.ylabel("Proporción")
plt.xlabel("Tiempo de viaje (horas)")
plt.xticks(rotation=0)
plt.legend(title="Target")
plt.show()

In [ ]:
# Calcular los conteos de cada valor de target por grupo etario
conteos_df = personas.groupby('grupo_viaje_tarde')['target'].value_counts().unstack(fill_value=0)

# Graficar como barras apiladas
conteos_df.plot(kind='bar', stacked=False)
plt.title("Cantidad de personas por tiempo de viaje y target")
plt.ylabel("Cantidad")
plt.xlabel("Tiempo de viaje (horas)")
plt.legend(title="Target")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()